In [1]:
import sys
sys.path.append("../v3-polars")

%load_ext autoreload
%autoreload 2

from v3 import state
import numpy as np

import polars as pl

from datetime import date, timedelta, datetime, timezone
import os

from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
WETH = ['0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'.lower(), 
        '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'.lower(),
        '0x4200000000000000000000000000000000000006'.lower()]

In [3]:
starting = datetime(year = 2023, month = 3, day = 1)

address = '0xc31e54c7a869b9fcbecc14363cf510d1c41fa443'
arb = state.v3Pool(address, 'arbitrum')#, update = True)
priceArb = arb.getPriceSeries(starting, frequency = '2h', gas = True)

address = '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640'
eth = state.v3Pool(address, 'ethereum')#, update = True)
priceEth = eth.getPriceSeries(starting, frequency = '2h', gas = True)

address = '0x85149247691df622eaf1a8bd0cafd40bc45154a9'
op = state.v3Pool(address, 'optimism')#, update = True)
priceOp = op.getPriceSeries(starting, frequency = '2h', gas = True)

PARTITIONED DS: estimated cardinality: 0.9991552 exceeded the boundary: 0.4, running default HASH AGGREGATION
PARTITIONED DS: estimated cardinality: 1 exceeded the boundary: 0.4, running default HASH AGGREGATION
PARTITIONED DS: estimated cardinality: 1 exceeded the boundary: 0.4, running default HASH AGGREGATION
PARTITIONED DS: estimated cardinality: 1 exceeded the boundary: 0.4, running default HASH AGGREGATION
PARTITIONED DS: estimated cardinality: 0.9980997 exceeded the boundary: 0.4, running default HASH AGGREGATION
PARTITIONED DS: estimated cardinality: 1 exceeded the boundary: 0.4, running default HASH AGGREGATION


In [4]:
def getLiqByBps(bps, bn, pool, normalize = False):
    distance = bps * 100
    
    try:
        swapDf, values = pool.calcSwapDF(bn)
    except Exception as e:
        return pl.DataFrame()
    
    tick = (values[-1] // pool.ts) * pool.ts
    
    high, low = tick + distance, tick - distance

    # i want USDC so i ask for the token 
    # that is not ETH
    invert = False
    if pool.token1 in WETH:
        invert = True

    if normalize:
        col = 'tick_normalized'
    else:
        col = 'tick'
        
    dis = (swapDf
         .filter((pl.col('tick_a') < high) &
                      (pl.col('tick_a') >= low))
         .with_columns(tick_normalized = pl.col('tick_a') - tick)
         .with_columns(tick_adj = (-1 * pl.col(col) if invert else pl.col(col)))
         .select([pl.col('tick_adj'), pl.col('liquidity')])
        )
   
    return dis 

In [ ]:
iterator = [*priceArb.iter_rows()] #unpack to use tqdm

tgt_cnt = len(iterator) // 2 + 100
cnt = 0

arbDis = pl.DataFrame()
ethDis = pl.DataFrame()
opDis = pl.DataFrame()

bps = 5
missed_op = 0

for (ts, l2BN, _, gas_price, gas_used) in tqdm(iterator):
#     cnt+=1
#     if cnt > 5_000:
#         break
        
    if ts >= pd.to_datetime("2024-01-01").to_pydatetime().replace(tzinfo = timezone.utc):
        continue
        
    startingAmtIn = 1e6
    ethRow = priceEth.filter(pl.col('block_timestamp') == ts)
    opRow = priceOp.filter(pl.col('block_timestamp') == ts)
    
    if opRow.is_empty():
        missed_op+=1
        continue
    
    # -- arbitrum
    dis = getLiqByBps(bps, l2BN, arb, normalize = True)
    if dis.is_empty():
        continue
        
    # normalize
    dis = (dis
           .with_columns(liquidity = 100 * (pl.col('liquidity') - pl.col('liquidity').min()) / 
                                  (pl.col('liquidity').max() - pl.col('liquidity').min())
                        )
          )
    
    if arbDis.is_empty():
        arbDis = dis
    else:
        arbDis.extend(dis)
        
    # -- op
    dis = getLiqByBps(bps, opRow['block_number'].item(), op, normalize = True)
    if dis.is_empty():
        continue
        
    dis = (dis
           .with_columns(liquidity = 100 * (pl.col('liquidity') - pl.col('liquidity').min()) / 
                                  (pl.col('liquidity').max() - pl.col('liquidity').min())
                        )
          )
    if opDis.is_empty():
        opDis = dis
    else:
        opDis.extend(dis)
    
    # -- eth
    dis = getLiqByBps(bps, ethRow['block_number'].item(), eth, normalize = True)
    if dis.is_empty():
        continue
        
    dis = (dis
           .with_columns(liquidity = 100 * (pl.col('liquidity') - pl.col('liquidity').min()) / 
                                  (pl.col('liquidity').max() - pl.col('liquidity').min())
                        )
          )
    if ethDis.is_empty():
        ethDis = dis
    else:
        ethDis.extend(dis)

  9%|███▎                                  | 389/4530 [07:10<1:21:57,  1.19s/it]

In [ ]:
# save data to use in R
arbDis.write_csv("graphs/data/arb.csv")
ethDis.write_csv("graphs/data/eth.csv")
opDis.write_csv("graphs/data/op.csv")

In [ ]:
# sample plot to make sure things work
fig, ax = plt.subplots()

data = arbDis.group_by('tick_adj').agg(pl.col('liquidity').quantile(.5)).sort(pl.col("tick_adj"))
ax.plot(100 * (1.0001 ** data['tick_adj'] - 1), data['liquidity'], label = 'Arbitrum', color = 'dodgerblue')

data = ethDis.group_by('tick_adj').agg(pl.col('liquidity').quantile(.5)).sort(pl.col("tick_adj"))
ax.plot(100 * (1.0001 ** data['tick_adj'] - 1), data['liquidity'], label = 'Ethereum', color = 'forestgreen')

data = opDis.group_by('tick_adj').agg(pl.col('liquidity').quantile(.5)).sort(pl.col("tick_adj"))
ax.plot(100 * (1.0001 ** data['tick_adj'] - 1), data['liquidity'], label = 'Optimism', color = 'firebrick')

ax.set_ylabel("Median Normalized Concentration")
ax.set_xlabel("Percent away from mid")
ax.legend()

fig.savefig("avg_liq_concentration.png")